In [ ]:
from components.importacao_diretorios_windows import listagem_pastas, listagem_arquivos,listagem_arquivos_downloads, pega_nome
from components.extract_text_pdf import extract_text_pdf
from components.importacao_automacao_excel_pandas import carrega_arquivo
from components.importacao_caixa_dialogo import DialogBox
from components.checar_ativacao_google_drive import checa_google_drive
from components.configuracao_db import configura_db, ler_sql
from components.configuracao_selenium_drive import configura_selenium_driver
from components.enviar_emails import enviar_email_com_anexos
import tkinter as tk
import mysql.connector
from re import search
from pathlib import Path
from shutil import copy
from openpyxl import load_workbook
import win32com.client as win32
from dotenv import load_dotenv
import os
from time import sleep, time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import  NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [134]:
particao = 'H'
mes = '01'
ano = '2024'

planilha_vales_sst = Path(f"{particao}:\\Meu Drive\\Relatorio_Vales_Saude_Seguranca\\{mes}-{ano}\\Relatorio_Vales_Saude_Seguranca_{mes}.{ano}.xlsx")
planilha_reembolsos = Path(f"{particao}:\\Meu Drive\\Relatorio_Boletos_Salario_Reembolso\\{mes}-{ano}\\Relatorio_Boletos_Salario_Reembolso.xlsx")

centro_custo = 'Torquato Clinica Odontologica S S LTDA-0001'

In [132]:
df_vales_sst = pd.read_excel(planilha_vales_sst, usecols='C:H', skiprows=1)
df_vales_sst

,CLIENTE,Vale Transporte,Assinatura Eletronica,Vale Refeição,Ponto Eletrônico,Saúde/Segurança do Trabalho
0,ITAPERUNA,NaN,NaN,NaN,NaN,NaN
1,Condominio Residencial Bela Aurora,0.00,0.0,0.0,0.0,208.10
2,EDIFICIO RESIDENCIAL CASABLANCA,0.00,0.0,0.0,0.0,208.40
3,CENTRO COMERCIAL E EMPRESARIAL ITAPERUNA - C.C...,0.00,0.0,0.0,0.0,208.40
4,Condominio Edificio Centro Clinico,0.00,0.0,0.0,0.0,148.51
5,Condominio Residencial Redentor,0.00,0.0,0.0,0.0,106.89
6,Obra Herbert Pinto Pena,0.00,0.0,0.0,0.0,187.41
7,Condominio Residencial Village del Rio,0.00,0.0,0.0,0.0,166.48
8,POPMED CLINICA MEDICA E ODONTOLOGICA LTDA,0.00,0.0,0.0,45.5,166.48
9,LUARE MIO CONFECÇOES LTDA,0.00,0.0,0.0,327.5,0.00


In [135]:
resultado = df_vales_sst.loc[df_vales_sst['CLIENTE'] == centro_custo, ['Vale Transporte', 'Assinatura Eletronica', 'Vale Refeição', 'Ponto Eletrônico', 'Saúde/Segurança do Trabalho']]
vale_transporte = str(resultado['Vale Transporte'].values[0]).replace("R$", "").replace(",", ".")
assinat_eletronica = str(resultado['Assinatura Eletronica'].values[0]).replace("R$", "").replace(",", ".")
vale_refeicao = str(resultado['Vale Refeição'].values[0]).replace("R$", "").replace(",", ".")
ponto_eletronico = str(resultado['Ponto Eletrônico'].values[0]).replace("R$", "").replace(",", ".")
sst = str(resultado['Saúde/Segurança do Trabalho'].values[0]).replace("R$", "").replace(",", ".")
print(vale_transporte, assinat_eletronica, vale_refeicao, ponto_eletronico, sst)

234.0 0.0 0.0 78.0 466.8


In [136]:
df_reembolsos = pd.read_excel(planilha_reembolsos, usecols='B:D', skiprows=1)
reembolsos = df_reembolsos[(df_reembolsos['CLIENTE'] == centro_custo) & (df_reembolsos['Descrição'].notnull()) & (df_reembolsos['Valor'].notnull())]
descricao_reembolsos = reembolsos['Descrição'].tolist()
valores_reembolsos = reembolsos['Valor'].tolist()
descricao_reembolsos, valores_reembolsos

([], [])